In [1]:
import numpy as np
import pandas as pd 
import csv
import nltk
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize


import os,sys,re

stopwords = nltk.corpus.stopwords.words('english')
CustomListofWordstoExclude = ["'s",'say','says','said','s',"n't"] 
stopwords.extend(CustomListofWordstoExclude)


In [2]:

df = pd.read_csv(r'D:\.vscode\misinformation_detection\fake_and_real_news_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4594 entries, 0 to 4593
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   idd     4594 non-null   object
 1   title   4593 non-null   object
 2   text    4594 non-null   object
 3   label   4594 non-null   object
dtypes: object(4)
memory usage: 143.7+ KB


In [3]:
aa = 0
i=0
similarity_array = []

for i in range(len(df['text'])):
    
    aa+=1
    data_text = df['text'].values[i]
    data_title = df['title'].values[i]

    X_list = word_tokenize(str(data_title))  
    Y_list = word_tokenize(str(data_text))
 
    sw = stopwords
    l1 =[];l2 =[]
    X_set = {w for w in X_list if not w in sw}  
    Y_set = {w for w in Y_list if not w in sw} 
  
    rvector = X_set.union(Y_set)
    
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula  
    for i in range(len(rvector)): 
        c+= l1[i]*l2[i]
    try:
        cosine = c/float((sum(l1)*sum(l2))**0.5) 
    except ZeroDivisionError:
        cosine = 0
    
    similarity_array.append(cosine)
    
print("value of aa:")
print(aa)

similarity_array.insert(0,"stance")

value of aa:
4594


In [4]:
with open(r'D:\.vscode\misinformation_detection\fake_and_real_news_dataset.csv', 'r', encoding='utf-8', errors='replace') as in_csv, \
     open(r'D:\.vscode\misinformation_detection\news_dataset2.csv', 'w', newline='', encoding='utf-8') as out_csv:
    
    reader = csv.reader(in_csv)
    writer = csv.writer(out_csv)

    # Loop through each row in the input CSV
    for row, new_col in zip(reader,similarity_array):
        row.append(new_col)
        writer.writerow(row)

In [7]:
dff = pd.read_csv(r'D:\.vscode\misinformation_detection\news_dataset2.csv')
dff["stance"].info


result = [] 
for value in dff["stance"]: 
  
    if float(value) >= 0.06: 
        result.append("match") 
    else: 
        result.append("nomatch") 
       
dff["Result"] = result

dff.to_csv( r'.\news_dataset.csv' , index=False)
dff.head(5)


,idd,title,text,label,stance,Result
0,Fq+C96tcx+,‘A target on Roe v. Wade ’: Oklahoma bill maki...,UPDATE: Gov. Fallin vetoed the bill on Friday....,REAL,0.157060,match
1,bHUqK!pgmv,Study: women had to drive 4 times farther afte...,Ever since Texas laws closed about half of the...,REAL,0.182210,match
2,4Y4Ubf%aTi,"Trump, Clinton clash in dueling DC speeches","Donald Trump and Hillary Clinton, now at the s...",REAL,0.123542,match
3,_CoY89SJ@K,Grand jury in Texas indicts activists behind P...,A Houston grand jury investigating criminal al...,REAL,0.124015,match
4,+rJHoRQVLe,"As Reproductive Rights Hang In The Balance, De...",WASHINGTON -- Forty-three years after the Supr...,REAL,0.041135,nomatch


In [8]:
dff = pd.read_csv(r".\news_dataset.csv")

In [9]:
dff.head()

,idd,title,text,label,stance,Result
0,Fq+C96tcx+,‘A target on Roe v. Wade ’: Oklahoma bill maki...,UPDATE: Gov. Fallin vetoed the bill on Friday....,REAL,0.157060,match
1,bHUqK!pgmv,Study: women had to drive 4 times farther afte...,Ever since Texas laws closed about half of the...,REAL,0.182210,match
2,4Y4Ubf%aTi,"Trump, Clinton clash in dueling DC speeches","Donald Trump and Hillary Clinton, now at the s...",REAL,0.123542,match
3,_CoY89SJ@K,Grand jury in Texas indicts activists behind P...,A Houston grand jury investigating criminal al...,REAL,0.124015,match
4,+rJHoRQVLe,"As Reproductive Rights Hang In The Balance, De...",WASHINGTON -- Forty-three years after the Supr...,REAL,0.041135,nomatch
